In [1]:
import os
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import use_hyperas
from tuner import net
from tuner.dataset import ClassificationDataset, AugmentDataset

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from easydict import EasyDict
params = EasyDict({
    'src_dir': '../micin-dataset/brain',
    'dst_dir': './brain'
})

### Convert  dataset from brain-dir to ready-dir 

In [3]:
df = load_data.df_fromdir_brain(params.src_dir)

In [4]:
load_data.classed_dir_fromdf(df, "tmp/brain")

### Load dataset via ready-dir

In [5]:
brain = ClassificationDataset('tmp/brain')

In [6]:
brain_dataset = brain

In [7]:
min(brain_dataset.counts_train_data().values())

46

In [8]:
aug_brain = AugmentDataset(brain_dataset)

Initialised with 246 image(s) found.
Output directory set to /home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train/output.

In [9]:
aug_brain.augmented_dir

'/home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/auged'

In [10]:
aug_brain.dataset.train_dir

'/home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train'

### Search best condition of data augmentation

In [11]:
aug_brain.search_opt_augment(model=net.neoaug)

def data():
    train_dir = '/home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train' 
    test_dir = '/home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/validation'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0   MS140.jpg    MS  MS/MS140.jpg   
1   MS162.jpg    MS  MS/MS162.jpg   
2   MS068.jpg    MS  MS/MS068.jpg   
3   MS124.jpg    MS  MS/MS124.jpg   
4   MS059.jpg    MS  MS/MS059.jpg   
5   PD043.jpg    PD  PD/PD043.jpg   
6   PD114.jpg    PD  PD/PD114.jpg   
7   PD092.jpg    PD  PD/PD092.jpg   
8   PD002.jpg    PD  PD/PD002.jpg   
9   PD096.jpg    PD

Epoch 5/10
 - 0s - loss: 1.3307 - acc: 0.3616 - val_loss: 8.5675 - val_acc: 0.4500
Epoch 6/10
 - 0s - loss: 1.3546 - acc: 0.3661 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 7/10
 - 0s - loss: 1.3479 - acc: 0.3661 - val_loss: 9.5594 - val_acc: 0.3500
Epoch 8/10
 - 0s - loss: 1.3487 - acc: 0.3616 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 9/10
 - 0s - loss: 1.3385 - acc: 0.3973 - val_loss: 9.0588 - val_acc: 0.4000
Epoch 10/10
 - 0s - loss: 1.3454 - acc: 0.3750 - val_loss: 9.9302 - val_acc: 0.3500
Test accuracy: 0.34999999404
--------------------------------------------------------------------------------
Operations: 3
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: RandomErasing (probability=0.5 rectangle_area=0.2 )
	2: Shear (probability=0.3 max_shear_left=2 max_shear_right=2 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.
Epoch 1/10
 - 0s - loss: 1.4434 - acc: 0.3393 - val_loss: 9.8870 - val_acc: 

Epoch 1/10
 - 1s - loss: 1.4053 - acc: 0.3214 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 2/10
 - 0s - loss: 1.3190 - acc: 0.4330 - val_loss: 8.0865 - val_acc: 0.5000
Epoch 3/10
 - 0s - loss: 1.3239 - acc: 0.4018 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 4/10
 - 0s - loss: 1.2948 - acc: 0.4018 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/10
 - 0s - loss: 1.2094 - acc: 0.5045 - val_loss: 8.8655 - val_acc: 0.4500
Epoch 6/10
 - 0s - loss: 1.1779 - acc: 0.5179 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
 - 0s - loss: 1.1050 - acc: 0.5804 - val_loss: 7.2745 - val_acc: 0.5500
Epoch 8/10
 - 0s - loss: 0.9875 - acc: 0.6429 - val_loss: 9.6709 - val_acc: 0.4000
Epoch 9/10
 - 0s - loss: 0.9295 - acc: 0.6562 - val_loss: 6.6840 - val_acc: 0.5500
Epoch 10/10
 - 0s - loss: 0.9274 - acc: 0.6384 - val_loss: 7.0986 - val_acc: 0.5500
Test accuracy: 0.550000011921
--------------------------------------------------------------------------------
Operations: 2
	0: Flip (probability=0.5 top_bottom_left_

### Execute data augmentation with best condition

In [12]:
aug_brain.augment_dataset()

Processing MS125.jpg:   4%|▍         | 7/184 [00:00<00:03, 54.26 Samples/s]

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train/MS/output.

Processing PD044.jpg:   3%|▎         | 5/184 [00:00<00:05, 32.90 Samples/s]  

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train/PD/output.

Processing N080.jpg:  10%|▉         | 18/184 [00:00<00:01, 98.88 Samples/s]  

Initialised with 79 image(s) found.
Output directory set to /home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train/N/output.

Processing PS068.jpg:   3%|▎         | 5/184 [00:00<00:05, 32.35 Samples/s] 

Initialised with 75 image(s) found.
Output directory set to /home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/train/PS/output.

### Tuning hyper parameter of CNN

In [13]:
best_condition, best_model = use_hyperas.exec_hyperas(
    aug_brain.augmented_dir,
    aug_brain.dataset.validation_dir,
    net.simplenet,
    batch_size = 32,
    epochs=10,
    optimizer='adam'
)
fname = 'simplenet.hdf5'
best_model.save(fname)

def data():
    train_dir = '/home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/auged' 
    test_dir = '/home/fytroo/Tuner/test/standard_datasets/5a66e0f7b037cb29b22f3106/validation'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0   MS140.jpg    MS  MS/MS140.jpg   
1   MS162.jpg    MS  MS/MS162.jpg   
2   MS068.jpg    MS  MS/MS068.jpg   
3   MS124.jpg    MS  MS/MS124.jpg   
4   MS059.jpg    MS  MS/MS059.jpg   
5   PD043.jpg    PD  PD/PD043.jpg   
6   PD114.jpg    PD  PD/PD114.jpg   
7   PD092.jpg    PD  PD/PD092.jpg   
8   PD002.jpg    PD  PD/PD002.jpg   
9   PD096.jpg    PD

Test accuracy: 0.25
Train on 736 samples, validate on 20 samples
Epoch 1/10
 - 1s - loss: 12.1300 - acc: 0.2418 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 2/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 3/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 4/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 5/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 6/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 7/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 8/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 9/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 10/10
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Test accuracy: 0.25
Train on 736 samples, validate on 20 samples
Epoch 1/10
 - 1s - 